In [1]:
import itertools

def get_input(day):
    filename = 'input/day{}.txt'.format(day)
    with open(filename) as f:
        return [line.strip() for line in f]

# [Day 1: Inverse Captcha](http://adventofcode.com/2017/day/1)

## Part I

**Task:** Given a circular sequence of digits, find the sum of all digits that match the next digit in the list.

**Examples**:
* `1122` -> `3`
* `91212129` -> `9`

In [2]:
def solve_captcha(sequence):
    total = 0
    previous_number = sequence[-1]
    for number in sequence:
        if previous_number == number:
            total += int(previous_number)
        previous_number = number
    return total

day1_input = get_input(1)[0]
solve_captcha(day1_input)

1341

## Part II

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. The list has an even number of elements.

**Example:**

* `1212` -> `6`
* `123123` -> `12`

In [3]:
def solve_halfway_captcha(sequence):
    total = 0
    for index in range(len(sequence) // 2):
        first_num = int(sequence[index])
        second_num = int(sequence[index + len(sequence) // 2])
        if first_num == second_num:
            total += first_num + second_num
    return total    
    
solve_halfway_captcha(day1_input)

1348

# [Day 2 - Corruption Checksum](http://adventofcode.com/2017/day/2)

## Part I

**Task:** Given a spreadsheet of rows and numbers, calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

**Example:**
```
5 1 9 5
7 5 3
2 4 6 8
```
The checksum is `8 + 4 + 6 = 18`.

In [4]:
def calculate_checksum(spreadsheet):
    return sum([max(row) - min(row) for row in spreadsheet])
    
def build_spreadsheet(rows):
    return [list(map(int, row.split('\t'))) for row in rows]
    
spreadsheet = build_spreadsheet(get_input(2))
calculate_checksum(spreadsheet)

50376

## Part II

Now, instead of calculating the difference between the max and min, the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. Find those numbers on each line, divide them, and add up each line's result.

**Example:**
```
5 9 2 8
9 4 7 3
3 8 6 5
```
The checksum is `4 + 3 + 2 = 9`.

In [5]:
def calculate_division_checksum(spreadsheet):
    total = 0
    for row in spreadsheet:
        total += [b // a for a, b in itertools.combinations(sorted(row), 2) if b % a == 0][0]
    return total

calculate_division_checksum(spreadsheet)

267